In [2]:
import numpy as np
import torch
import os
from torch.utils import data as Data
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.optimizer import Optimizer
from torch.utils.data.dataloader import DataLoader
from dataprocess import MyData,Tokenizer
from transformer import Transformer
from tqdm import tqdm
import math
import sys


In [3]:
!pwd

/Users/xiaochen/code/NLP/MT


In [4]:
#GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#数据路径
data_path = "./data/"
bpevocab_path = data_path+"bpevocab.txt"
train_de = data_path+"train_de.txt"
train_en = data_path+"train_en.txt"
valid_de = data_path+"valid_de.txt"
valid_en = data_path+"valid_en.txt"
print("file path:"+valid_en)
testloss = 0
batch_size = 512
lr = 0.0001
dict_number = 10148
epochs = 15
n_layers = 6
seq_len = 32
heads =4
d_model = 512
norm_shape = [seq_len, d_model]
#创建数据
train_data = MyData(train_de, train_en, bpevocab_path)
train_data = Data.DataLoader(train_data, shuffle=True, batch_size=batch_size, num_workers=0, pin_memory=True)
valid_data = MyData(valid_de, valid_en, bpevocab_path)
valid_data = Data.DataLoader(valid_data, shuffle=False, batch_size=batch_size, num_workers=0, pin_memory=True)

# 创建网络
model = Transformer(n_layers, dict_number, seq_len, heads, d_model, norm_shape)
model.load_state_dict(torch.load("../de2en_14.pkl", map_location=torch.device(device)))


cpu
file path:./data/valid_en.txt
from token list(len(10144),make a dict(len(10148)))
from token list(len(10144),make a dict(len(10148)))


<All keys matched successfully>

In [9]:
sen1_en = "and now we would like to introdu@@ ce you to jo@@ ey ."
sen1_de = "jetzt würden wir euch gerne jo@@ ey vorstellen ."
sen2_en = "so today , i &apos;m going to tell you about some people who didn &apos;t move out of their neighbor@@ ho@@ ods ."
sen2_de = "also , heute werde ich ihnen von ein paar menschen erzählen , die nie aus ihrer hei@@ mat@@ stadt weg@@ gezogen sind ."
sen3_en = "and he ended up achie@@ ving at a level that had never before been con@@ tem@@ pl@@ ated by any member of his family ."
sen3_de = "und am ende schaff@@ te er es auf ein bildungs@@ nive@@ au , das niemand in seiner familie je zuvor in erwä@@ gung gezogen hatte ."
sen_en = [sen1_en, sen2_en, sen3_en]
sen_de = [sen1_de, sen2_de, sen3_de]
token = Tokenizer()
src_seq = token.encoder_sentence(sen_de)
src_seq = token.padding(src_seq)
src_seq = torch.IntTensor(src_seq)
print(src_seq.shape)
tgt_seq = token.encoder_sentence(sen_en)
tgt_seq = token.padding(tgt_seq)
tgt_seq = torch.IntTensor(tgt_seq)
print(tgt_seq[0])


from token list(len(10144),make a dict(len(10148)))
torch.Size([3, 32])
tensor([   2,    7,   95,   26,  160,   87,   10, 6119,  577,   23,   10, 1071,
        2404,    5,    3,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], dtype=torch.int32)


In [42]:

outputs_seq = model(src_seq, tgt_seq)
oseq = outputs_seq.max(dim=-1).indices
print(oseq.shape)
out_sen = token.decoder_sentence(outputs_seq.max(dim=-1).indices.tolist())
out_sen1 = " ".join(out_sen[0])
out_sen2 = " ".join(out_sen[1])
out_sen3 = " ".join(out_sen[2])
print(out_sen1)
print(sen1_en)
print(out_sen2)
print(sen2_en)

torch.Size([3, 32])
i in that 50 spring income dna meaning blue spring street is &#93; policy <eos> , <eos> , and by <eos> , , give to a the of . &apos;s war and
and now we would like to introdu@@ ce you to jo@@ ey .
, gets dna source — fully security — changing gas office — star — known capacity complicated bor@@ <eos> glo@@ order street oil fish &apos;s we the new the . given and
so today , i &apos;m going to tell you about some people who didn &apos;t move out of their neighbor@@ ho@@ ods .
